In [1]:
import pandas as pd
import json
from generate_graph import get_propositions, generateEdges, createGraph, get_propositions_nosplit
from refine_graph import refine
from query_graph import QueryGraph
from tqdm import tqdm
tqdm.pandas()

df = pd.read_json('datasets/HotPotQA/hotpot_dev_distractor_v1.json')

## Experiment


### Graph creation and refinement section

To run the evaluation script

```
python hotpot_evaluate_v1.py <path_to_prediction> <path_to_gold>
```

Example:

```
python hotpot_evaluate_v1.py <sample_dev_pred.json> <hotpot_dev_fullwiki_v1.json>
```

In [ ]:
context_list = df['context']

In [ ]:
df.head(11)

In [ ]:
df.iloc[7]['context']

In [ ]:
context_list[7:11]

In [ ]:
propositions = []

for i, value in enumerate(tqdm(context_list[7:11])):
        for j, context in enumerate(value):
                index = 1
                if index < len(context):
                    words = context[1]
                    combined = " ".join(words)
                    get_propositions_nosplit(combined, propositions)

In [ ]:
# Open the file in read mode
with open('propositions.txt', 'r') as file:
    # Read all lines and store them in a list
    propositions_from_file = [line.strip() for line in file]

print(propositions_from_file)

In [ ]:
list_of_edges = generateEdges(propositions_from_file)

In [ ]:
if createGraph(list_of_edges):
    print("Success")
else:
    print("Failed")

In [ ]:
qg = QueryGraph()

if refine(qg, "person"):
    print("yes")
else:
    print("no")

### Q and A section

In [ ]:
# def find_supporting_facts(question, answer, context):
#     supporting_facts = []
#     for passage in context:
#         title, sentences = passage
#         for i, sentence in enumerate(sentences):
#             if answer in sentence or any(word in sentence for word in question.split()):
#                 supporting_facts.append([title, i])
#     return supporting_facts

In [2]:
qg = QueryGraph()
my_dict = {}

# Define a function to apply to each row
def print_qa(row):
    
    question = row['question']
    questionid = row['_id']
    req = qg.get_requirements(question)
    result = qg.answer_question(question, req.content)

    if result is not None:
    
        model_answer = result['result']

        if "don't know the answer" in model_answer:
            model_answer = ""
    
        if len(row['answer']) > 0:
            real_answer = row['answer']
        else:
            real_answer = "\"\""

        if len(result['intermediate_steps']) > 0:
            cypher_query = result['intermediate_steps'][0]['query']
            context = result['intermediate_steps'][1]['context']
       
            if(len(context) < 1):
            #     result = refine_query(cypher_query[6:], question)
            #     model_answer = result['result'] 
                model_answer = "\"\""
                my_dict.update({questionid: ""})
            else:
                my_dict.update({questionid: model_answer})
                
        with open("preds.txt", "a") as preds:
            preds.write("question: " + question + "\n")
            preds.write("real_answer: " + real_answer + "\n")
            preds.write("model_answer: " + model_answer + "\n")  
            preds.write("cypher_query: " + cypher_query + "\n")
            preds.write("======================" + "\n")  
        preds.close()

    else: 
        my_dict.update({questionid: ""})
        

In [3]:
my_dict = {}
# Apply the function to each row
df[0:10].progress_apply(print_qa, axis=1)
    
with open("dev_fullwiki_pred.json", "a") as json_file:
    json.dump(my_dict, json_file, indent=4)

  0%|          | 0/10 [00:00<?, ?it/s]



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (w)-[r1]-(x)-[r2]-(y)
WHERE (
    toLower(r1.metadata) CONTAINS 'scott derrickson' OR 
    toLower(r1.metadata) CONTAINS 'nationality'
)
AND (
    toLower(r2.metadata) CONTAINS 'ed wood' OR 
    toLower(r2.description) CONTAINS 'nationality'
)
RETURN DISTINCT w,x,y
LIMIT 100
Full Context:
[]


 20%|██        | 2/10 [00:03<00:14,  1.83s/it]


> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (w)-[r1]-(x)-[r2]-(y)
WHERE (
    toLower(r1.metadata) CONTAINS 'kiss and tell' OR 
    toLower(r1.metadata) CONTAINS 'corliss archer'
)
AND (
    toLower(r2.metadata) CONTAINS 'government position' OR 
    toLower(r2.description) CONTAINS 'government position'
)
RETURN DISTINCT w,x,y
LIMIT 100
Full Context:
[]


 30%|███       | 3/10 [00:07<00:18,  2.69s/it]

...


> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (w)-[r1]-(x)-[r2]-(y)
WHERE (
    toLower(r1.metadata) CONTAINS 'annie morton' OR 
    toLower(r1.metadata) CONTAINS 'birthdate'
)
AND (
    toLower(r2.metadata) CONTAINS 'terry richardson' OR 
    toLower(r2.description) CONTAINS 'birthdate'
)
RETURN DISTINCT w,x,y
LIMIT 100
Full Context:
[{'w': {'name': 'Tsi-La Organics', 'label': 'Object'}, 'x': {'name': 'Annie Morton', 'label': 'Person'}, 'y': {'name': 'Terry Richardson', 'label': 'Person'}}, {'w': {'name': 'advocate for natural healthcare', 'label': 'Miscellaneous'}, 'x': {'name': 'Annie Morton', 'label': 'Person'}, 'y': {'name': 'Terry Richardson', 'label': 'Person'}}, {'w': {'name': 'advocate for organic lifestyle choices', 'label': 'Miscellaneous'}, 'x': {'name': 'Annie Morton', 'label': 'Person'}, 'y': {'name': 'Terry Richardson', 'label': 'Person'}}, {'w': {'name': 'vegetarian', 'label': 'Miscellaneous'}, 'x': {'name': 'Annie Morton', 'lab

100%|██████████| 10/10 [00:46<00:00,  5.33s/it]


> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (w)-[r1]-(x)-[r2]-(y)
WHERE (
    toLower(r1.metadata) CONTAINS 'local h' OR 
    toLower(r1.metadata) CONTAINS 'for against'
)
AND (
    toLower(r2.metadata) CONTAINS 'united states' 
)
RETURN DISTINCT w,x,y
LIMIT 100
Full Context:
[{'w': {'name': 'American rock band', 'label': 'Miscellaneous'}, 'x': {'name': 'Local H', 'label': 'Object'}, 'y': {'name': "Hallelujah! I'm A Bum", 'label': 'Object'}}, {'w': {'name': 'Scott Lucas', 'label': 'Person'}, 'x': {'name': 'Local H', 'label': 'Object'}, 'y': {'name': "Hallelujah! I'm A Bum", 'label': 'Object'}}, {'w': {'name': 'Matt Garcia', 'label': 'Person'}, 'x': {'name': 'Local H', 'label': 'Object'}, 'y': {'name': "Hallelujah! I'm A Bum", 'label': 'Object'}}, {'w': {'name': 'Joe Daniels', 'label': 'Person'}, 'x': {'name': 'Local H', 'label': 'Object'}, 'y': {'name': "Hallelujah! I'm A Bum", 'label': 'Object'}}, {'w': {'name': 'John Sparkman', 'label': 'Pe

100%|██████████| 10/10 [00:49<00:00,  4.98s/it]


> Finished chain.


In [3]:
x = df.iloc[5]
question = x['question']
print("question:", question)
answer = x['answer']
print("answer:", answer)

qg = QueryGraph()
req = qg.get_requirements(question)
res = qg.answer_question(question, req.content)
res

question: 2014 S/S is the debut album of a South Korean boy group that was formed by who?
answer: YG Entertainment


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (w)-[r1]-(x)-[r2]-(y)
WHERE (
    toLower(r1.metadata) CONTAINS '2014 s/s' OR 
    toLower(r1.metadata) CONTAINS 'debut album'
)
AND (
    toLower(r2.metadata) CONTAINS 'south korean boy group' OR 
    toLower(r2.description) CONTAINS 'formed by'
)
RETURN DISTINCT w,x,y
LIMIT 100
Full Context:
[{'w': {'name': '2014 S/S', 'label': 'Event'}, 'x': {'name': 'YG Entertainment', 'label': 'Object'}, 'y': {'name': 'Winner', 'label': 'Miscellaneous'}}, {'w': {'name': 'Mad Town', 'label': 'Miscellaneous'}, 'x': {'name': 'Madtown', 'label': 'Miscellaneous'}, 'y': {'name': 'South Korean boy group', 'label': 'Miscellaneous'}}]

> Finished chain.


{'query': '2014 S/S is the debut album of a South Korean boy group that was formed by who?',
 'requirements': '1. Key components of the question:\n   - The debut album "2014 S/S"\n   - The South Korean boy group associated with the album\n   - The entity or individual responsible for forming the boy group\n\n2. Requirements needed to answer the question:\n   - Name of the South Korean boy group\n   - Name of the individual or entity that formed the boy group\n\n3. Additional context or data needed:\n   - None',
 'result': 'YG Entertainment',
 'intermediate_steps': [{'query': "MATCH (w)-[r1]-(x)-[r2]-(y)\nWHERE (\n    toLower(r1.metadata) CONTAINS '2014 s/s' OR \n    toLower(r1.metadata) CONTAINS 'debut album'\n)\nAND (\n    toLower(r2.metadata) CONTAINS 'south korean boy group' OR \n    toLower(r2.description) CONTAINS 'formed by'\n)\nRETURN DISTINCT w,x,y\nLIMIT 100"},
  {'context': [{'w': {'name': '2014 S/S', 'label': 'Event'},
     'x': {'name': 'YG Entertainment', 'label': 'Object'

## Graph refinement 

In [ ]:
qg = QueryGraph()

QUERY = """
MATCH (N) RETURN N
"""
res = qg._graph.query(QUERY)
res

In [ ]:
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings

In [ ]:
neo4j_url = "bolt://localhost:7687"
neo4j_user = "neo4j"
neo4j_password = "12345678"

Create embeddings

In [ ]:
# # Now we initialize from existing graph
# existing_graph = Neo4jVector.from_existing_graph(
#     embedding=OpenAIEmbeddings(model='text-embedding-3-small'),
#     url=neo4j_url,
#     username=neo4j_user,
#     password=neo4j_password,
#     index_name="person_index",
#     node_label="Person",
#     text_node_properties=["name"],
#     embedding_node_property="embedding",
# )

Refactor node labels

In [ ]:
# # cq = """
# # MATCH (en:Entity)
# # WHERE en.label = 'Person'
# # WITH collect(en) AS persons
# # CALL apoc.refactor.rename.label("Entity", "Person", persons)
# # YIELD batches, total, timeTaken, committedOperations
# # RETURN batches, total, timeTaken, committedOperations;
# # """
# # res = qg._graph.query(cq)

Create projection

In [ ]:
cq = """
MATCH (p:Person)
RETURN gds.graph.project(
  'personproj',
  p,
  null,
  {
    sourceNodeProperties: p { .embedding },
    targetNodeProperties: {}
  }
)
"""

res = qg._graph.query(cq)

Use KNN to measure node similarities

In [ ]:
cq = """
CALL gds.knn.stream('projection2', {
topK: 1,
nodeProperties: ['embedding'],
// The following parameters are set to produce a deterministic result
randomSeed: 1337,
concurrency: 1,
sampleRate: 1.0,
deltaThreshold: 0.0
})
YIELD node1, node2, similarity
WHERE similarity > .90
RETURN gds.util.asNode(node1).name AS Person1, gds.util.asNode(node2).name AS Person2, similarity
ORDER BY similarity DESCENDING, Person1, Person2
"""

res = qg._graph.query(cq)
res

In [ ]:
for i in res:
    for key, value in i.items():
        print(f"{key}: {value}")

In [ ]:
def are_dictionaries_equivalent(d1, d2):
    # Check if both have the same keys
    if d1.keys() != d2.keys():
        return False

    # Check if Person1 and Person2 are swapped but equivalent
    if (d1['Person1'] == d2['Person2'] and d1['Person2'] == d2['Person1'] and d1['similarity'] == d2['similarity']):
        return True
    
    return False


In [ ]:
newlist = res

In [ ]:
for idx, val in enumerate(res):   
    for idx2, val2 in enumerate(newlist):
        if val != val2:
            if are_dictionaries_equivalent(val, val2):
                newlist.pop(idx2)


In [ ]:
unique = []

for value in newlist:
    if value not in unique:
        unique.append(value)

unique

In [ ]:
node_name = "Shirley Temple"

query = f"""
MATCH (n:Person {{name: '{node_name}'}})
RETURN n
"""
print(query)

res = qg._graph.query(query)
res


Merge similar nodes (Entity resolution)

In [ ]:
for idx, val in enumerate(unique):
    print(val)
    p1 = val['Person1']
    p2 = val['Person2']    
    cq = f"""
    MATCH (a1:Person {{name: '{p1}'}}), (a2:Person {{name: '{p2}'}})
    WITH head(collect([a1,a2])) as nodes
    CALL apoc.refactor.mergeNodes(nodes,{{
    properties:"discard",
    mergeRels:true
    }})
    YIELD node
    RETURN node;
    """
    res = qg._graph.query(cq)
    res